In [ ]:
from keras.models import load_model
from firebase import firebase
import cv2
import numpy as np 
import argparse
from PIL import Image
import imutils

sample_anamoly_video1 = "path to testing video"
sample_anamoly_video2 = "path to testing video"
sample_normal_video = "path to testing video"

def mean_squared_loss(x1,x2):
    difference = x1-x2
    a,b,c,d,e = difference.shape
    n_samples = a*b*c*d*e
    sq_difference = difference**2
    Sum = sq_difference.sum()
    distance = np.sqrt(Sum)
    mean_distance = distance/n_samples

    return mean_distance

def smart_cam(sample_video):
    threshold = 0.00057
    model = load_model("Xmodel.h5")

    cap = cv2.VideoCapture(sample_video)
    print("SMART SURVAILLANCE: {}".format(cap.isOpened()))

    while cap.isOpened():
        imagedump = []
        _, frame = cap.read()

        if frame.any()== None:
            break

        for i in range(10):
            _, frame = cap.read()
            image = imutils.resize(frame,width=700,height=600)

            frame = cv2.resize(frame, (227,227), interpolation = cv2.INTER_AREA)
            gray = 0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
            gray = (gray-gray.mean())/gray.std()
            gray = np.clip(gray,0,1)
            imagedump.append(gray)

        imagedump = np.array(imagedump)

        imagedump.resize(227,227,10)
        imagedump = np.expand_dims(imagedump,axis=0)
        imagedump = np.expand_dims(imagedump,axis=4)

        output = model.predict(imagedump)

        loss = mean_squared_loss(imagedump,output)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break

        # print(loss)

        if loss > threshold:
            cv2.putText(image,"Abnormal Event",(100,80),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
            if loss < 0.00058 and loss > 0.00056: 
                cv2.putText(image,"Severity: Moderate",(100,120),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            elif loss > 0.00062:
                cv2.putText(image,"Severity: High",(100,120),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)

        cv2.imshow("video",image)

    cap.release()
    cv2.destroyAllWindows()
    
smart_cam(sample_normal_video)
smart_cam(sample_anamoly_video1)
smart_cam(sample_anamoly_video2)


In [ ]:
# YOLO people count output anamoly detection 
from keras.models import load_model
from firebase import firebase
import cv2
import numpy as np 
import argparse
from PIL import Image
import imutils

sample_normal_video = "path to testing video"
    
def env_check(anamoly_count, people_count):
    if anamoly_count < 5 and people_count > 10:
        return 'SAFE'
    elif anamoly_count > 5 and people_count > 10:
        return 'MODERATE'
    elif anamoly_count > 10 and people_count < 10:
        return 'DANGER'
    
def mean_squared_loss(x1,x2):
    difference = x1-x2
    a,b,c,d,e = difference.shape
    n_samples = a*b*c*d*e
    sq_difference = difference**2
    Sum = sq_difference.sum()
    distance = np.sqrt(Sum)
    mean_distance = distance/n_samples

    return mean_distance

def smart_cam(sample_video):
    anamoly_count = 0
    threshold = 0.00057
    model = load_model("Xmodel.h5")

    cap = cv2.VideoCapture(sample_video)
    print("SMART SURVAILLANCE: {}".format(cap.isOpened()))

    while cap.isOpened():
        imagedump = []
        _, frame = cap.read()

        if frame.any()== None:
            break

        for i in range(10):
            _, frame = cap.read()
            image = imutils.resize(frame,width=700,height=600)
            people_count = 11
            frame = cv2.resize(frame, (227,227), interpolation = cv2.INTER_AREA)
            gray = 0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
            gray = (gray-gray.mean())/gray.std()
            gray = np.clip(gray,0,1)
            imagedump.append(gray)

        imagedump = np.array(imagedump)

        imagedump.resize(227,227,10)
        imagedump = np.expand_dims(imagedump,axis=0)
        imagedump = np.expand_dims(imagedump,axis=4)

        output = model.predict(imagedump)

        loss = mean_squared_loss(imagedump,output)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            env = env_check(anamoly_count, people_count)
            print('Total {0} Anamoly detected, People Count: {1}: {2}'.format(anamoly_count, people_count, env))
            break

        if loss > threshold:
            if loss < 0.00060 and loss > 0.00056: 
                anamoly_count += 1
                cv2.putText(image,"Abnormal Event",(100,80),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
                cv2.putText(image,"Severity: Moderate",(100,120),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            elif loss > 0.00065:
                anamoly_count += 1
                cv2.putText(image,"Abnormal Event",(100,80),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
                cv2.putText(image,"Severity: High",(100,120),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)

        cv2.imshow("video",image)
        
    
    cap.release()
    cv2.destroyAllWindows()
    

smart_cam(sample_normal_video)
